In [ ]:
import warnings
from helper import *
from pprint import pprint

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
file_dict, file_dir, master_dir = get_data_files()

In [ ]:
master_df = pd.DataFrame()

In [ ]:
# get the file indexes
available_file_indexes = [{a: [c.split("_")[0][2:] for c in sorted(b)]} for a, b in file_dict.items()]
available_file_indexes = {k: v for d in available_file_indexes for k, v in d.items()}

In [ ]:
# sort the file_dict by the file name number
for year, files in file_dict.items():
    file_dict[year] = sorted(files, key=lambda x: int(x.split("_")[0][2:]))

In [ ]:
# find top 20 pipes for every file
for year, files in file_dict.items():
    access_index = 0
    for index in available_file_indexes[year]:
        master_df = find_common_pipes(file_index=access_index, file_year=year, top_level_df=master_df,
                                      file_dict=file_dict, master_dir=master_dir, threshold=50)
        access_index += 1

In [ ]:
# transpose the dataframe
master_df_T = master_df.copy().T
master_df_T.columns, master_df_T.loc["X", :] = master_df_T.loc[["X"], :].values[0], master_df_T.columns

In [ ]:
# create multilevel columns
exp_df = pd.DataFrame(columns=pd.MultiIndex.from_product([master_df_T.columns, ["Pipe TTNr", "Total"]]).unique())

In [ ]:
# add the data to the multilevel columns
for i in range(len(master_df_T)):
    exp_df.loc[i, :] = master_df_T.iloc[i, :].values

In [ ]:
# drop the unnecessary index
exp_df = exp_df.copy().drop(index=0, inplace=False)

In [ ]:
exp_df

In [ ]:
# create two sheets in the Excel file
with pd.ExcelWriter(file_dir, engine="openpyxl", mode="w") as writer:
    exp_df.to_excel(writer, sheet_name='General')
    exp_df.T.to_excel(writer, sheet_name='Experimental')

In [ ]:
# create multi-level index
three_level_columns = create_three_level_index(df=exp_df)

# # set the multi-level index
exp_df_th = exp_df.copy()
exp_df_th.columns = pd.MultiIndex.from_tuples(three_level_columns)

In [ ]:
# format the Excel files
format_general_sheet(file_dir=file_dir)
format_experimental_sheet(file_dir=file_dir)

In [ ]:
create_bar_plot(df=exp_df_th,
                selected_year=2021,
                file_index="16",
                ascending=True,
                threshold=20)

In [ ]:
exp_df.head()

In [ ]:
len(exp_df.iloc[:, 0])

In [ ]:
pipe_codes = exp_df.iloc[:, ::2].values.flatten()
unique_pipes = np.array(list(set(pipe_codes)))

In [ ]:
vertical_df = pd.DataFrame()

for i in range(0, len(exp_df.columns), 2):
    temp = exp_df.copy().iloc[:, i:i + 2]
    temp.columns = ['Pipe TTNr', 'Total']
    vertical_df = pd.concat([vertical_df, temp], axis=0, ignore_index=True)

In [ ]:
# get the rows with unique pipes
unique_pipe_rows = vertical_df["Pipe TTNr"].unique()

In [ ]:
# add the total column for the rows with same Pipe TTNr
for i in range(len(unique_pipe_rows)):
    vertical_df.loc[vertical_df["Pipe TTNr"] == unique_pipe_rows[i], "Total"] = \
        vertical_df.loc[vertical_df["Pipe TTNr"] == unique_pipe_rows[i], "Total"].sum()

In [ ]:
# drop duplicate Pipe TTNrs
final_df = vertical_df.drop_duplicates(subset="Pipe TTNr",
                                       keep="first",
                                       inplace=False,
                                       ignore_index=True).sort_values(by="Total", ascending=False).copy()
final_df = final_df.reset_index(drop=True, inplace=False).copy()

In [ ]:
final_df['Pipe TTNr'] = final_df['Pipe TTNr'].astype(str)

In [ ]:
final_df.loc[final_df["Pipe TTNr"].apply(lambda x: x.isnumeric()), :].copy()

In [ ]:
unique_pipe_bar_plot(pipe_df=final_df,
                     total_quantity_limit=50000,
                     fig_size=(16, 7),
                     rotation='vertical',
                     ascending=True)

In [ ]:
pipes_dict = {}

for i in range(0, len(exp_df.columns), 2):
    temp = exp_df.copy().iloc[:, i:i + 2]
    pipes_dict[temp.columns[0][0]] = temp[(temp.columns[0][0], 'Pipe TTNr')].unique()

pprint(pipes_dict)

In [ ]:
# count the number of occurrences for each pipe and which weeks it occurs
pipe_occurrences, week_occurrences = {}, {}

for key, value in pipes_dict.items():
    for pipe in value:
        if pipe in pipe_occurrences.keys():
            pipe_occurrences[pipe] += 1
            week_occurrences[pipe].append(key)
        else:
            pipe_occurrences[pipe] = 1
            week_occurrences[pipe] = [key]

merged_dict = {k: [pipe_occurrences[k], week_occurrences[k]] for k in pipe_occurrences.keys()}

In [ ]:
# create a dataframe from the dictionary
pipe_occurrences_df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['Occurrences', 'Weeks'])
pipe_occurrences_df.sort_values(by="Occurrences", ascending=False, inplace=False)

In [ ]:
pipes_in_70_df = pipe_occurrences_df.loc[pipe_occurrences_df["Occurrences"] > 70, :].sort_values(by="Occurrences",
                                                                                                 ascending=False)
pipes_in_70_df

In [ ]:
pipes_in_70_df = pipes_in_70_df.reset_index(inplace=False).copy()
pipes_in_70_df = pipes_in_70_df.rename(columns={"index": "Pipe TTNr"}, inplace=False).copy()

In [ ]:
pipes_in_70 = pipes_in_70_df['Pipe TTNr'].values
pipes_in_70

In [ ]:
all_in_one = pd.DataFrame()

In [ ]:
for year, plan in file_dict.items():
    for file in plan:
        df = pd.read_excel(f'{master_dir}/{str(year)}/{file}', sheet_name='Pivot')
        df = df.fillna(0)
        df.iloc[:, 5:26] = df.iloc[:, 5:26].apply(pd.to_numeric, errors='coerce')
        df['Total'] = df.iloc[:, 5:26].sum(axis=1)
        df.loc["Hat", "Total"] = 0
        df = df.sort_values(by=['Total'], inplace=False, ascending=False, ignore_index=True).copy()

        aa = df.iloc[:, [1, -1]].copy()
        aa.index.name = aa.columns[0].split(" ")[3]
        aa.columns = ['Pipe TTNr', aa.columns[-1]]

        aa = aa.loc[aa['Pipe TTNr'].isin(pipes_in_70), :].copy()
        aa.reset_index(inplace=True)
